## 不会发生异常

### 1. `run_until_complete()` 直接运行函数

```py
import asyncio


async def comain():
    raise SystemExit("SystemExit")


def main():
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(comain())
    except BaseException as e:
        print("caught {}".format(e))
    finally:
        loop.close()


if __name__ == "__main__":
    main()
```

### task 捕获异常后 cancel()

```py
import asyncio


async def comain():
    raise SystemExit("SystemExit")


def main():
    loop = asyncio.get_event_loop()
    task = loop.create_task(comain())
    try:
        loop.run_until_complete(task)
    except BaseException as e:
        print("caught {}".format(e))
        task.cancel()
    finally:
        loop.close()


if __name__ == "__main__":
    main()
```

### 用asyncio.wait()去执行

```py
asyncio.wait
```

Task 或者 Future 在**运行中发生错误**，**并且在其内部没有被捕获**，必须在使其执行的地方进行 `cancle()`取消，否则会在`Loop`中抛出异常未回收的错误。但是不影响`Loop`中其他代码的执行。

有如下场景：

```py
import asyncio


async def comain():
    for i in range(-2, 2):
        print(1 / i)


async def task():
    t1 = loop.create_task(comain())
    t2 = loop.create_task(comain())
    t3 = loop.create_task(comain())


if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(task())
    except BaseException as e:
        print("caught: {}".format(e))
    finally:
        loop.close()
    print('其他')
```


即使捕获到了异常，依然会抛出错误信息，`Task exception was never retrieved' 根据提示，我们应该另需处理 Task 中的异常，可以直接将异常的Task或者 Future取消执行即可，cancel()

```py
import asyncio


async def comain():
    raise SystemExit("SystemExit")


def main():
    loop = asyncio.get_event_loop()
    task = loop.create_task(comain())
    try:
        loop.run_until_complete(task)
    except BaseException as e:
        print("caught {}".format(e))
        # 取消执行
        task.cancel()
    finally:
        loop.close()


if __name__ == "__main__":
    main()
```